In [1]:
import pandas as pd 
import numpy as np
import time 

df = pd.read_parquet('../scripts/df_merged1.parquet', engine='pyarrow')

In [2]:
df.shape

(5796, 10)

In [3]:
unique_ids = df['objectId'].unique().tolist()

In [4]:
condition = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [5]:
df[df['source'] == 0]

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_weight,nr_weight
2,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.0,0.0
3,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
5,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
6,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59083,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.0,0.0
7,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59083,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5784,ZTF18abxavej,1356446886315015012,2.459111e+06,59105,1,0.001210,0.000079,0.000962,1.113912e-19,0,0.0,0.0
5785,ZTF18abxavej,1356446886315015012,2.459111e+06,59105,2,0.001272,0.000018,0.000994,2.219433e-19,0,0.0,0.0
5787,ZTF18abxavej,1356446886315015012,2.459111e+06,59106,2,0.001272,0.000018,0.000994,2.219433e-19,0,0.0,0.0
5790,ZTF18abxavej,1356446886315015012,2.459111e+06,59108,1,0.001210,0.000079,0.000962,1.113912e-19,0,0.0,0.0


In [6]:
df[['source','dc_weight']].head()

,source,dc_weight
0,1,1.353949e+09
1,1,2.615609e+08
2,0,0.000000e+00
3,0,0.000000e+00
4,1,2.325079e+08


In [7]:
len(unique_ids)

100

In [8]:
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids}
#print(d)

count = 0

for k in unique_ids:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        #print(len(s), len(s_n1))
        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        #alpha = s[::2] + s[1::2] 
        #print(s)
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

ZTF18acevrat :  [2.72223802e-29 6.60467896e-29 1.72424973e-28 3.45576088e-28
 3.40134284e-28 0.00000000e+00 0.00000000e+00 4.25159071e-28
 1.72261550e-28 1.21136515e-28 2.65502893e-28 0.00000000e+00
 1.65446204e-28 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.49826529e-28 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.52460487e-29 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00] 

ZTF18abnvapv :  [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.23952226e-29
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.12819566e-29
 0.00000000e+00 0.00000000e+00 1.06286671e-29 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.87730117e-30 1.42468172e-29
 0.00000000e+00 7.04650499e-30 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00] 

ZTF17aaadyyl :  [0.00000000e+00 0.00000000e+00

In [ ]:
(d['ZTF17aaaeclk']<= factor)

In [ ]:
d['ZTF17aaaeclk']

In [ ]:
Q

In [ ]:
df[df['objectId']=='ZTF17aaaeclk']

In [ ]:
alpha

In [ ]:
1.32646186*np.array((0.000286,0.000636))

#

# with l>0

In [19]:
start_time = time.time()



m=1
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1

chunk_size = 2 * (m + 1)
# Calculate the number of chunks
num_chunks = len(df[['dc_flux']]) // chunk_size
# Split the DataFrame into chunks and store them in an array
Q = [df[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

if m>0: #for j in range(1, max(1, m+1)): this isn't faster
    for j in range(1, m+1):
        Q.extend([df[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])

Q = pd.concat(Q) # here we transform Q from list of dataframes to a single data frame  
# we will be able to distinct the values by using l variable s

Number_l = (len(Q)) // (2*(m+1)) 

d = {k: [0] * Number_l for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['dc_flux', 'dc_weight']]
    n = len(T)
    f = T['dc_flux'].values
    w = T['dc_weight'].values
    
    zeros = np.zeros(n, dtype=float)

    for l in range(Number_l):
        #print(l)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        d[k][l]=0
        
        for j in range(0,m+1): 
            h = np.tile(Q[2*l*(m+1) +j*2: 2*l*(m+1) + j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         

        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[2*l*(m+1) + j*2: 2*l*(m+1) +j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            # 2*l*(m+1) this part is to take from Q by l 
            
            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k][l] = dd[::2] + dd[1::2]
            
            
        if (d[k][l] <= factor).all():
            print(k, ": ", d[k][l],"\n")
            count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

95
Elapsed time: 7.218683958053589 seconds


In [11]:
Number_l, len(Q), len(df)

(2897, 11588, 5796)

In [22]:
d[k]#[1]

[array([7.14627572e+00, 7.31440191e+00, 0.00000000e+00, 4.14265832e+01,
        4.93030954e+01, 4.53510082e-29, 9.76648760e-01, 4.60302234e+02,
        5.34815210e+02, 4.17174691e-02, 3.76050700e-01, 9.62917322e-01,
        6.43971175e+01, 1.08536980e+02, 9.31337749e+01, 8.85188695e+01,
        8.05268082e+01, 9.32757075e+01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.00301046e-30, 3.64130817e+01, 4.47804528e+01, 0.00000000e+00,
        5.29678375e+02, 6.42739787e+02, 6.87645497e+01]),
 array([7.14627572e+00, 6.97194099e+00, 0.00000000e+00, 4.14265832e+01,
        4.44674594e+01, 4.53510082e-29, 4.64278153e-01, 4.59810900e+02,
        5.08015128e+02, 4.17174691e-02, 3.76050700e-01, 9.62917322e-01,
        6.43971175e+01, 1.00657761e+02, 8.66169700e+01, 8.11693389e+01,
        7.62717156e+01, 8.58611888e+01, 7.35273331e-30, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [26]:
d.keys()

dict_keys(['ZTF17aaaeclk', 'ZTF17aaarimh', 'ZTF18acevrat', 'ZTF18abhxxas', 'ZTF18aakgeyw', 'ZTF18acoyqvk', 'ZTF17aactufx', 'ZTF18abcoyzc', 'ZTF18abnyhra', 'ZTF18abmwqzf'])

###### just to organize my ideas,  we now have all necessary implementations:

- 'd' is a function of 'l', 'i', and 'k'. However, I think it's impractical in its current form and requires modification.
- 'Q' is a function of 'l' and utilizes vectorized versions of 'hr' and 'hg'. It contains all possible 'l' values.

- 'T' of course depends on 'k'. While it might be possible to vectorize it, but it seems it will be too much and it's inusual i think)

###### 
 'Alpha' ? | | , I think it should be defined similarly to 'd'.

the problem is in the time taken by the code.
we can reduce the complexity by reduction the 'Q' for each step (I still need to understand the step after the test ) 


#

# TEST

In [ ]:
m=0
for j in range(1, max(1, m+1)):
    print(j*2)

if m>0: 
    for j in range(1, m+1):
        print(j*2)

In [ ]:
import timeit

m = 100

code_snippet_1 = """
for j in range(1, max(1, m+1)):
    j*2
"""

code_snippet_2 = """
if m > 0:
    for j in range(1, m+1):
        j*2
"""

time_taken_1 = timeit.timeit(stmt=code_snippet_1, number=1000000, globals=globals())
time_taken_2 = timeit.timeit(stmt=code_snippet_2, number=1000000, globals=globals())

print("Time snippet 1:", time_taken_1)
print("Time snippet 2:", time_taken_2)



#

#

#

# Test for Q all possible values ! 

In [ ]:
import pandas as pd

values = list(range(10))  

# Duplicate each value
duplicated_values = [val for val in values for _ in range(2)]


df2 = pd.DataFrame({'dc_flux': duplicated_values})

print(df2)

In [ ]:
# Define the size of each chunk
m=1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = len(df2[['dc_flux']]) // chunk_size

# Split the DataFrame into chunks and store them in an array
Q = [df2[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

Q

In [ ]:
for j in range(1, m+1):
    Q.extend([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
    #print([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
Q



In [ ]:
O = pd.concat(Q)

# Display the resulting DataFrame
print(O)

In [ ]:
m=1
(len(O)) // (2*(m+1))

In [ ]:
m=2
j=2
l=0
#np.tile(Q[(j*2):(j*2+2)], (4 // 2, 1)).ravel()
O[2*l*(m+1) +(j*2) : 2*l*(m+1) +(j*2+2)]

#

#

#

#

# distance dict to dataframe 

In [ ]:
d

In [27]:
list_of_dicts = []

# Iterate through the data dictionary and create dictionaries for each object ID
for object_id, distances in d.items():
    for distance in distances:
        list_of_dicts.append({'objectId': object_id, 'distance': distance})

# Create DataFrame from the list of dictionaries
pdf = pd.DataFrame(list_of_dicts)

# Display the DataFrame
print(pdf)

           objectId                                           distance
0      ZTF17aaaeclk  [0.0, 0.0, 0.0, 0.22820866720155883, 308.85099...
1      ZTF17aaaeclk  [0.23398952931368616, 0.0, 0.0, 0.228208667201...
2      ZTF17aaaeclk  [1.2756513794961397, 0.0, 6.8327929512214424e-...
3      ZTF17aaaeclk  [6.333078663371891, 0.0, 6.8327929512214424e-3...
4      ZTF17aaaeclk  [9.884293394402125, 0.0, 0.0, 21.5511690022007...
...             ...                                                ...
28965  ZTF18abmwqzf  [10.253066973985812, 10.248627233249975, 0.0, ...
28966  ZTF18abmwqzf  [10.253066973985812, 10.256977144825564, 0.0, ...
28967  ZTF18abmwqzf  [10.253066973985812, 10.269498662979752, 0.0, ...
28968  ZTF18abmwqzf  [12.592923754992654, 10.30539677906164, 0.0, 8...
28969  ZTF18abmwqzf  [10.308840275543563, 10.248627233249975, 0.0, ...

[28970 rows x 2 columns]


In [28]:
len(df)/2898

2.0

In [29]:
pdf['is_distance_gt_factor'] = pdf['distance'] <= factor

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [30]:
pdf

,objectId,distance
0,ZTF17aaaeclk,"[0.0, 0.0, 0.0, 0.22820866720155883, 308.85099..."
1,ZTF17aaaeclk,"[0.23398952931368616, 0.0, 0.0, 0.228208667201..."
2,ZTF17aaaeclk,"[1.2756513794961397, 0.0, 6.8327929512214424e-..."
3,ZTF17aaaeclk,"[6.333078663371891, 0.0, 6.8327929512214424e-3..."
4,ZTF17aaaeclk,"[9.884293394402125, 0.0, 0.0, 21.5511690022007..."
...,...,...
28965,ZTF18abmwqzf,"[10.253066973985812, 10.248627233249975, 0.0, ..."
28966,ZTF18abmwqzf,"[10.253066973985812, 10.256977144825564, 0.0, ..."
28967,ZTF18abmwqzf,"[10.253066973985812, 10.269498662979752, 0.0, ..."
28968,ZTF18abmwqzf,"[12.592923754992654, 10.30539677906164, 0.0, 8..."


# Trash

In [ ]:
df[df['objectId'] =='ZTF18acevrat' ]

In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [ ]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d0 = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])
                s = np.where(s_d == 0, 0, s_n / s_d)

                alpha += s
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d0[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d0[k] <= factor).all():
        print(d0[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       